In [61]:

from models.ConstraintAutoRec import ConstraintAutoRec
import utils.common
import evaluation
import importlib

import numpy as np
importlib.reload(utils.common)
importlib.reload(evaluation)
# model = ConstraintAutoRec(utils.common.movie_lens['dimensions'], epochs=30, novelty_weight=0.25, diversity_weight=0)
# model.train(utils.common.load_dataset(utils.common.movie_lens, 'train'), utils.common.movie_lens['train']['records'])


<module 'evaluation' from '/mnt/e/owncloud/FH/MasterThesis/Code/evaluation.py'>

In [62]:

predictions, actual = evaluation.evaluate(model, utils.common.movie_lens)


Accuracy: 0.3457339844872163
Precision: 1.0
Recall: 0.3457339844872163
MSE 0.6702403997788664
Diversity@5 0.37348606069855156
Diversity@10 0.37900318415666373
EPC@10 0.9847232771798894
EPD@10 0.3764372736013949


In [3]:
diversity_model = ConstraintAutoRec(utils.common.movie_lens['dimensions'], epochs=30, novelty_weight=0.25, diversity_weight=2.0)
diversity_model.train(utils.common.load_dataset(utils.common.movie_lens, 'train'), utils.common.movie_lens['train']['records'])


Epoch 1/30
19/19 [==============================] - 13s 684ms/step - loss: 0.2508 - accuracy: 0.0103
Epoch 2/30
19/19 [==============================] - 0s 22ms/step - loss: 0.0996 - accuracy: 0.0954
Epoch 3/30
19/19 [==============================] - 0s 20ms/step - loss: 0.0961 - accuracy: 0.1353
Epoch 4/30
19/19 [==============================] - 0s 19ms/step - loss: 0.1018 - accuracy: 0.1119
Epoch 5/30
19/19 [==============================] - 1s 26ms/step - loss: 0.0957 - accuracy: 0.0592
Epoch 6/30
19/19 [==============================] - 0s 20ms/step - loss: 0.1115 - accuracy: 0.0879
Epoch 7/30
19/19 [==============================] - 0s 22ms/step - loss: 0.0961 - accuracy: 0.0946
Epoch 8/30
19/19 [==============================] - 0s 22ms/step - loss: 0.0992 - accuracy: 0.0933
Epoch 9/30
19/19 [==============================] - 0s 22ms/step - loss: 0.0758 - accuracy: 0.0789
Epoch 10/30
19/19 [==============================] - 0s 24ms/step - loss: 0.0689 - accuracy: 0.0905
Epoch 1

In [58]:
pred_2, actual = evaluation.evaluate(diversity_model, utils.common.movie_lens)

Accuracy: 0.3812122953174375
Precision: 1.0
Recall: 0.3812122953174375
MSE 0.6687574924015608
Diversity@5 0.3556821688031748
Diversity@10 0.36323029734955137
EPC@10 0.9779377030362106


In [13]:
movie_features = np.load(utils.common.movie_lens['item_features'])['arr_0']

In [21]:

# top_100 = evaluation.recommend_top_n(pred_2, 100)
print(evaluation.diversity_at_k(predictions, 5, movie_features))
print(evaluation.diversity_at_k(pred_2, 5, movie_features))


[0.35384535 0.4221759  0.36133661 0.37903591 0.37158885 0.37909942
 0.37158885 0.34767615 0.3977605  0.3999889  0.39673332 0.37158885
 0.36312487 0.33879023 0.33802584 0.44754982 0.37506144 0.34250773
 0.35636298 0.37158885 0.34767615 0.3452513  0.34767615 0.32982079
 0.32850409 0.38061155 0.37158885 0.36213975 0.3711608  0.39816611
 0.3999889  0.38964961 0.34949324 0.34087444 0.38274409 0.40040078
 0.35562639 0.34767615 0.27175493 0.39314323 0.39137657 0.36133661
 0.34767615 0.37184981 0.37491748 0.37158885 0.3999889  0.37158885
 0.28344052 0.40071555 0.34775549 0.41920553 0.37158885 0.34036513
 0.3999889  0.3999889  0.37684953 0.34869716 0.37193355 0.37643507
 0.32861484]
[0.32797424 0.3538495  0.33527071 0.37704107 0.33569401 0.43368328
 0.33527071 0.33569401 0.38577694 0.40354093 0.37212231 0.33527071
 0.3489091  0.36130503 0.36840594 0.37748977 0.28783923 0.42504729
 0.37944574 0.29403785 0.33527071 0.36873184 0.36539341 0.37939101
 0.40134614 0.37402168 0.33569401 0.30167627 0.30

In [48]:
user_profile = actual[0].nonzero()[0]
user_profile

array([    1,     2,    10,    11,    47,    50,   110,   150,   230,
         272,   296,   318,   328,   350,   356,   480,   500,   586,
         597,   608,   750,   778,   780,   805,   858,   910,   912,
         913,   953,  1032,  1035,  1056,  1080,  1130,  1136,  1193,
        1199,  1203,  1206,  1221,  1230,  1237,  1247,  1250,  1258,
        1265,  1288,  1517,  1580,  1608,  1610,  1617,  1625,  1669,
        1682,  1721,  1784,  1923,  1968,  1969,  2052,  2081,  2146,
        2231,  2268,  2302,  2324,  2353,  2502,  2512,  2542,  2571,
        2599,  2617,  2692,  2706,  2762,  2788,  2858,  2882,  2918,
        2959,  2997,  3052,  3408,  3451,  3481,  3576,  3578,  4010,
        4011,  4022,  4027,  4226,  4306,  4427,  4679,  4966,  4973,
        4993,  5247,  5618,  5620,  5669,  6787,  6848,  6874,  7361,
        7919,  8022,  8228,  8374,  8922,  9586, 10226])

In [93]:

dataset = utils.common.load_testset(utils.common.movie_lens)
dataset = dataset.batch(16)



In [97]:
for value in dataset:
    held_back = value['held_back']
    # TODO
    break

TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  6,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 11, 11, 11

In [47]:

toystory = movie_features[0]
psycho = movie_features[2098]
toystory2 =movie_features[2769] 
# print(cosine_similarity(a, b))
print(1 - scipy.spatial.distance.cosine(toystory, psycho))
print(1 - scipy.spatial.distance.cosine(toystory, toystory2))

0.6159151078726672
0.9555216674015986


In [49]:

loaded = np.load(utils.common.movie_lens['item_features'], allow_pickle=True)

In [52]:
loaded['known_frequency']


array([[3.58824209e-01],
       [1.60606236e-01],
       [9.19534420e-02],
       ...,
       [3.61026470e-05],
       [7.22052941e-06],
       [7.22052941e-06]])